# 중요한 몇줄..

1. [/_search](http://localhost:9200/_search)를 하면 cluster안에 있는 모든 documents를 가져오게 됩니다. (주의 필요!)
2. In Elasticsearch, all data in every field is indexed by default. 
3. POST는 ID를 지정안해도 되고, PUT은 ID를 지정해야함.. (PUT는 특정 URL Resource에 넣으라는 뜻이고, POST는 만들라는 뜻이기 때문)

# Multi-Index, MultiType

| URL | Description |
|:----|:------------|
| /_search | Search all types in all indices |
| /gb/_search | Search all types in the gb index |
| /gb,us/_search | Search all types in the gb and us indices |
| /g*,u*/_search | Search all types in any indices beginning with g or beginning with u |
| /gb/user/_search | Search type user in the gb index |
| /gb,us/user,tweet/_search | Search types user and tweet in the gb and us indices|
| /_all/user,tweet/_search | Search types user and tweet in all indices |

# Pagination

* size: 몇건의 documents를 가져올지 정합니다.
* from: 해당 int만큼 앞부분을 skip합니다.

예를 들어서 한페이지에 5개씩 문서가 보이도록 만드려면, 다음과 같은 requests가 필요할 것입니다.

```
GET /_search?size=5
GET /_search?size=5&from=5
GET /_search?size=5&from=10
```

In [3]:
# -*- coding:utf-8 -*-
from elasticsearch import Elasticsearch
from pprint import pprint as pp
from datetime import datetime

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Index example data

[/us/tweet/_search](http://localhost:9200/us/tweet/_search)에서 확인가능합니다.<br>
이때 중요하게 볼 점은 **hints**안의  **total** 그리고 **max_score** 입니다.

In [5]:
es.index(index='us', doc_type='tweet', id=1, body={'date': datetime.now(), 
                                                   'name': 'John Smith',
                                                   'tweet': 'The Query DSL is really powerful and flexible'})

{u'_id': u'1',
 u'_index': u'us',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'tweet',
 u'_version': 2,
 u'created': False,
 u'result': u'updated'}

# Mapping

예를 들어서 [/us/_mapping/tweet/](http://localhost:9200/us/_mapping/tweet/) 에 들어가면 어떻게 Fields들이 정의되어 있는지 알 수 있습니다. <br>
따라서 date가 string으로 또는 date type으로 어떻게 해석되느냐에 따라서 검색 결과가 달라집니다.

In [15]:
pp(es.indices.get_mapping(index='us', doc_type='tweet'))

{u'us': {u'mappings': {u'tweet': {u'properties': {u'date': {u'type': u'date'},
                                                  u'name': {u'fields': {u'keyword': {u'ignore_above': 256,
                                                                                     u'type': u'keyword'}},
                                                            u'type': u'text'},
                                                  u'tweet': {u'fields': {u'keyword': {u'ignore_above': 256,
                                                                                      u'type': u'keyword'}},
                                                             u'type': u'text'}}}}}}


# Inverted Index 

Elasticsearch는 Inverted Index를 사용하여 full-text Search를 빠르게 처리합니다.<br>
Inverted Index는 a list of all the unique words로 구성되어 있습니다. 

예를 들어서 2개의 documents가 있으며, content field에 저장되어 있습니다. 

* The quick brown fox jumped over the lazy dog
* Quick brown foxes leap over lazy dogs in summer

Inverted Index를 만들기 위해서는, 각각 documents안의 content field를 단어들로 seperate해줍니다. (이때 각각 생성된 단어들은 terms 또는 tokens 이라고 부릅니다.) 그다음 terms 들을 모두 sort해주고, 각각의 documents에 나온대로 list를 만들어줍니다.

```
Term      Doc_1  Doc_2
-------------------------
Quick   |       |  X
The     |   X   |
brown   |   X   |  X
dog     |   X   |
dogs    |       |  X
fox     |   X   |
foxes   |       |  X
in      |       |  X
jumped  |   X   |
lazy    |   X   |  X
leap    |       |  X
over    |   X   |  X
quick   |   X   |
summer  |       |  X
the     |   X   |
------------------------
```

**quick brown**이라고 검색을 하면 다음과 같은 경과과 나오게 됩니다.

```
Term      Doc_1  Doc_2
-------------------------
brown   |   X   |  X
quick   |   X   |
------------------------
Total   |   2   |  1
```

Naive Similarity Algorithm (단순히 매칭되는 갯수로 판단) 을 사용하면 첫번재 Document가 더 query와 관련있다고 말할 수 있습니다. <br>
하지만 여기에는 몇가지 문제점이 있습니다.

1. Quick 그리고 quick은 서로 다른 terms입니다. 하지만 user는 같은 단어로 생각하고 query를 날렸을 것입니다.
2. fox 그리고 foxes는 매우 유사합니다. (dog 그리고 dogs처럼..) 이 단어들은 같은 **root word** 입니다.
3. jumped 그리고 leap은 같은 root word는 아니지만, 의미상 같은 의미입니다. (동의어)